In [2]:
import torch
from sbi.inference import MNPE
from sbi.utils import BoxUniform
import pandas as pd
import numpy as np
import pathlib 


from sklearn.neighbors import NearestNeighbors


import os
import json
import hashlib
from pathlib import Path
from datetime import datetime
from typing import Optional


import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt


In [69]:
import numpy as np
import pandas as pd
from pathlib import Path
import torch
from sklearn.neighbors import NearestNeighbors


def infer_relative_bin_edges(n_bins=6, r_min=1e-3, r_max=1.0):
    edges = np.geomspace(r_min, r_max, n_bins) #geomspace functoin returns numbers spaced evenly on a log scale
    edges = np.insert(edges, 0, 0.0) # you need to start at 0 but cannot with geom space because log 0 is -inf so you add it here (6 bins requires 7 edges)
    return edges

def relative_bin_fractions_from_events(depths_A, norm_depth_A, r_edges):
    x = np.asarray(depths_A, float)
    #make sure nothing funky is going on
    x = x[np.isfinite(x)]
    if x.size == 0 or norm_depth_A <= 0:
        return np.zeros(len(r_edges) - 1, float)

    
    #add the 1e-12 to avoid the divide by 0 error just in case 
    r = x / (norm_depth_A + 1e-12) 
    hist, _ = np.histogram(r, bins=r_edges) #build histogram
    hist[-1] += np.sum(r > r_edges[-1])  # overflow, finds all depts grater than last bin top edge tgeb adds to the last bin

    total = hist.sum() #adds all values in each bin of hist 
    if total == 0:
        return np.zeros_like(hist, dtype=float)

    return hist / total #final fraction



def compute_centered_track_asymmetry(x, eps=1e-12):
    x = np.asarray(x, dtype=float)
    if x.size == 0:
        return 0.0
    center = np.mean(x)
    n_left = np.sum(x < center)
    n_right = np.sum(x > center)
    total = n_left + n_right
    if total == 0:
        return 0.0
    return (n_right - n_left) / (total + eps)

def compute_centered_nn_asymmetry(x, z, eps=1e-12):
    x = np.asarray(x, float)
    z = np.asarray(z, float)
    if x.size < 2:
        return 0.0
    center = np.mean(x)
    left_mask = x < center
    right_mask = x > center
    left_pts = np.column_stack((x[left_mask], z[left_mask]))
    right_pts = np.column_stack((x[right_mask], z[right_mask]))
    def mean_nn(points):
        if points.shape[0] < 2:
            return np.nan
        nn = NearestNeighbors(n_neighbors=2).fit(points)
        dists, _ = nn.kneighbors(points)
        return float(np.mean(dists[:, 1]))
    d_left = mean_nn(left_pts)
    d_right = mean_nn(right_pts)
    if np.isnan(d_left) or np.isnan(d_right):
        return 0.0
    return (d_right - d_left) / (d_right + d_left + eps)

# The rest of your code (relative_bin_fractions_from_events, infer_relative_bin_edges) remains unchanged.
# The rest of your code (relative_bin_fractions_from_events, infer_relative_bin_edges) remains unchanged.
def preprocess2(data_path: str | Path, n_bins: int = 6):
    df = pd.read_csv(data_path)
    rename_map = {"x_ang": "x", "y_ang": "y", "z_ang": "z"}
    df = df.rename(columns={k: v for k, v in rename_map.items() if k in df.columns})
    required_cols = ["x", "y", "z", "ion_number", "parity"]
    for c in required_cols:
        if c not in df.columns:
            raise KeyError(f"Missing required column '{c}' in {data_path}")

    # Handle energy columns
    if "energy_keV" in df.columns:
        energy_keV = pd.to_numeric(df["energy_keV"], errors="coerce")
    elif "energy_eV" in df.columns:
        energy_keV = pd.to_numeric(df["energy_eV"], errors="coerce") / 1e3
    elif "energy" in df.columns:
        e_raw = pd.to_numeric(df["energy"], errors="coerce")
        # keep continuous values
        energy_keV = e_raw / 1e3 if np.nanmax(e_raw) > 3000 else e_raw
    else:
        raise KeyError("No energy column found.")

    # Ensure numeric types
    df["x"] = pd.to_numeric(df["x"], errors="coerce")
    df["y"] = pd.to_numeric(df["y"], errors="coerce")
    df["z"] = pd.to_numeric(df["z"], errors="coerce")
    df["ion_number"] = pd.to_numeric(df["ion_number"], errors="coerce")
    df["energy_keV"] = pd.to_numeric(energy_keV, errors="coerce")
    df["parity"] = pd.to_numeric(df["parity"], errors="coerce")

    df = df.dropna(subset=["x", "y", "z", "ion_number", "energy_keV", "parity"])
    df = df.reset_index(drop=True)

    print(f"[DEBUG] Unique parity values in raw data: {sorted(df['parity'].unique())}")

    # ---------------------------------------------
    # Continuous energy grouping (do NOT cast to int)
    # ---------------------------------------------
    df["energy_norm"] = df["energy_keV"].round(3)  # or round().astype(int)
    bin_edges = infer_relative_bin_edges(n_bins=n_bins)

    rows, x_obs_list, theta_list, track_ids = [], [], [], []

    for (E_val, ion_no, par), g in df.groupby(["energy_norm", "ion_number", "parity"], sort=False):
        # === Extract track coordinates ===
        x = np.asarray(g["x"].values, dtype=float)
        z = np.asarray(g["z"].values, dtype=float)
        if x.size == 0:
            continue

        # === ENERGY FEATURES (magnitudes) ===
        mean_depth = float(np.mean(np.abs(x)))        # extent of displacement
        max_depth = float(np.max(np.abs(x)))          # furthest excursion
        norm_depth = float(np.percentile(np.abs(x), 95))
        n_vac = int(x.size)
        rbin_fracs = relative_bin_fractions_from_events(np.abs(x), norm_depth, bin_edges)

        # === PARITY-SENSITIVE FEATURES (signed) ===
        frac_pos = float(np.mean(x > 0))
        frac_neg = 1.0 - frac_pos
        mean_x_signed = float(np.mean(x))
        x_std = x.std() + 1e-12
        skew_x = float(np.mean((x - x.mean())**3) / (x_std**3))
        signed_com = float(np.sum(np.sign(x) * np.abs(x)) / (np.sum(np.abs(x)) + 1e-12))
        corr_xz = float(np.corrcoef(x, z)[0, 1]) if x.size > 1 else 0.0

        # === ASYMMETRY FEATURES (around global zero, not local mean) ===
        asym_count = compute_centered_track_asymmetry(x)     # uses global zero
        asym_nn = compute_centered_nn_asymmetry(x, z)

        # === BUILD SUMMARY ROW ===
        tid = f"E{E_val:.3f}_ion{int(ion_no)}_p{int(par)}"
        rows.append({
            "track_id": tid,
            "energy_keV": float(E_val),
            "parity": int(par),

            # Energy features
            "mean_depth_A": mean_depth,
            "max_depth_A": max_depth,
            "vacancies_per_ion": n_vac,
            **{f"rbin_frac_{i+1}": rbin_fracs[i] for i in range(n_bins)},

            # Parity-sensitive
            "frac_pos": frac_pos,
            "mean_x_signed": mean_x_signed,
            "skew_x": skew_x,
            "signed_com": signed_com,
            "corr_xz": corr_xz,

            # Asymmetry features
            "asym_count_centered": asym_count,
            "asym_nn_centered": asym_nn,
        })

        # === COMBINE FEATURE VECTOR ===
        x_obs_list.append([
            mean_depth, max_depth, n_vac,
            *rbin_fracs,
            frac_pos, mean_x_signed, skew_x, signed_com, corr_xz,
            asym_count, asym_nn
        ])

        # === PARAMETER VECTOR (energy continuous, parity discrete) ===
        par_index = int(0 if par == -1 else 1)
        theta_list.append([float(E_val), float(par_index)])
        track_ids.append(tid)

    # Convert to tensors
    df_summary = pd.DataFrame(rows)
    x_obs = torch.tensor(np.array(x_obs_list, dtype=np.float32))
    theta = torch.tensor(np.array(theta_list, dtype=np.float32))

    # ✅ Force parity clean-up
    theta[:, 1] = (theta[:, 1] == 1).float()

    # --- Debug summary ---
    print(f"[DEBUG] Number of tracks created: {len(track_ids)}")
    print(f"[DEBUG] x_obs shape: {x_obs.shape}")
    print(f"[DEBUG] theta shape: {theta.shape}")
    print(f"[DEBUG] Unique energies (keV): {sorted(np.unique(theta[:, 0]))}")
    print(f"[DEBUG] Unique parities: {sorted(np.unique(theta[:, 1]))}")
    print(f"[DEBUG] Energy range: [{theta[:, 0].min():.1f}, {theta[:, 0].max():.1f}] keV")
    print(f"[DEBUG] First 10 theta samples (energy, parity):\n{theta[:10]}")

    if len(track_ids) < 50:
        print(f"[WARNING] Only {len(track_ids)} tracks found. MNPE needs at least 100–500 for stable training.")

    parity_vals = set(theta[:, 1].tolist())
    if not parity_vals.issubset({0.0, 1.0}):
        print(f"[WARNING] Parity contains unexpected values: {parity_vals}. Should be {{0.0, 1.0}}")

    return x_obs, theta, track_ids, {"rel_bin_edges": bin_edges}, df_summary

In [ ]:
x_obs, theta, track_ids, meta, df_summary = preprocess2(
    Path("/Users/cbharathulwar/Documents/Research/Walsworth/Code/SBI/srim-sbi/data/nov3srim/vacancies_with_parity_centered.csv"), n_bins=6
)


[DEBUG] Unique parity values in raw data: [-1, 1]
[DEBUG] Number of tracks created: 9200
[DEBUG] x_obs shape: torch.Size([9200, 16])
[DEBUG] theta shape: torch.Size([9200, 2])
[DEBUG] Unique energies (keV): [1.0, 2.5, 5.0, 7.5, 10.0, 15.0, 20.0, 25.0, 30.0, 35.0, 40.0, 45.0, 50.0, 55.0, 60.0, 65.0, 70.0, 75.0, 80.0, 85.0, 90.0, 95.0, 100.0]
[DEBUG] Unique parities: [0.0, 1.0]
[DEBUG] Energy range: [1.0, 100.0] keV
[DEBUG] First 10 theta samples (energy, parity):
tensor([[ 1.0000,  1.0000],
        [ 2.5000,  1.0000],
        [ 5.0000,  1.0000],
        [ 7.5000,  1.0000],
        [10.0000,  1.0000],
        [15.0000,  1.0000],
        [20.0000,  1.0000],
        [25.0000,  1.0000],
        [30.0000,  1.0000],
        [35.0000,  1.0000]])


In [71]:
from sbi.inference import MNPE
import torch

# Continuous param
energy = theta[:, [0]].float()
# Discrete param (must be 0/1 float)
parity = theta[:, [1]].float()


theta_mixed = torch.cat([energy, parity], dim=1)


# === Normalize energy before training ===
E_mean = theta[:, 0].mean()
E_std = theta[:, 0].std()
theta[:, 0] = (theta[:, 0] - E_mean) / E_std

# Save stats to use later
energy_stats = {"mean": E_mean, "std": E_std}
print(f"[INFO] Energy normalized: mean={E_mean:.3f}, std={E_std:.3f}")

# === Train the mixed neural posterior estimator ===
inference = MNPE(device="cpu", show_progress_bars=True)
_ = inference.append_simulations(theta, x_obs).train()

# === Build posterior ===
posterior = inference.build_posterior(sample_with="direct")




[INFO] Energy normalized: mean=46.130, std=31.534


/opt/homebrew/Caskroom/miniconda/base/envs/sbi/lib/python3.12/site-packages/sbi/neural_nets/net_builders/mixed_nets.py:260: UserWarning: The mixed neural density estimator assumes that x contains continuous data in the first n-k columns (e.g., reaction times) and categorical data in the last k columns (e.g., corresponding choices). If this is not the case for the passed `x` do not use this function.
  return _build_mixed_density_estimator(
/opt/homebrew/Caskroom/miniconda/base/envs/sbi/lib/python3.12/site-packages/sbi/neural_nets/net_builders/mixed_nets.py:163: UserWarning: Inferring num_categories from batch_x. Ensure all categories are present.
  discrete_net = build_categoricalmassestimator(


 Neural network successfully converged after 49 epochs.

In [81]:
import torch

# Pick any track index (say the 10th one)
i = 100 

# Extract that row's summary features
x_o = x_obs[i].unsqueeze(0)  # shape [1, dim_x]

samples = posterior.sample((1000,), x=x_o)

# === De-normalize energy back to keV ===
samples_denorm = samples.clone()
samples_denorm[:, 0] = samples[:, 0] * energy_stats["std"] + energy_stats["mean"]

mean_pred = samples_denorm.mean(0)
parity_pred = samples_denorm[:, 1].round().mode()[0].item()
print(f"True energy, parity: {theta[i].tolist()}")
print(f"Predicted energy mean: {mean_pred[0].item():.3f}")
print(f"Predicted parity (mode): {int(parity_pred)}")

  0%|          | 0/1000 [00:00<?, ?it/s]

True energy, parity: [-0.5115190744400024, 1.0]
Predicted energy mean: 27.445
Predicted parity (mode): 1


In [82]:
num_to_check = 200   # how many tracks to test
samples_per_track = 200  # posterior samples per x_o

n = min(num_to_check, len(theta))
correct_parity = 0
energy_errors = []

for i in range(n):
    x_o = x_obs[i].unsqueeze(0)

    # --- Sample posterior ---
    samples = posterior.sample((samples_per_track,), x=x_o)

    # === De-normalize energy back to keV ===
    samples_denorm = samples.clone()
    samples_denorm[:, 0] = samples[:, 0] * energy_stats["std"] + energy_stats["mean"]

    # --- Energy metrics ---
    true_energy = theta[i, 0].item() * energy_stats["std"] + energy_stats["mean"]  # also unnormalize truth
    pred_energy = samples_denorm[:, 0].mean().item()

    energy_diff = abs(pred_energy - true_energy) / max(true_energy, 1e-8) * 100
    energy_errors.append(energy_diff)

    # --- Parity ---
    true_parity = int(theta[i, 1].item())
    pred_parity = int(samples_denorm[:, 1].round().mode()[0].item())

    if pred_parity == true_parity:
        correct_parity += 1

# --- Results ---
parity_acc = correct_parity / n * 100
mean_energy_diff = sum(energy_errors) / n

print(f"Checked {n} tracks with {samples_per_track} posterior samples each")
print(f"Parity accuracy: {parity_acc:.2f}% ({correct_parity}/{n})")
print(f"Avg. energy % error: {mean_energy_diff:.2f}%")

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

...

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

Checked 200 tracks with 200 posterior samples each
Parity accuracy: 98.50% (197/200)
Avg. energy % error: 9.55%


In [78]:
import pandas as pd

df = pd.read_csv("/Users/cbharathulwar/Documents/Research/Walsworth/Code/SBI/srim-sbi/data/nov3srim/vacancies_with_parity_centered.csv")
print(df["energy"].unique())
print(df["parity"].unique())
print(df["ion_number"].nunique())

[  1.    2.5   5.    7.5  10.   15.   20.   25.   30.   35.   40.   45.
  50.   55.   60.   65.   70.   75.   80.   85.   90.   95.  100. ]
[ 1 -1]
200
